In [1]:
!pip install cvxopt
! pip install pyfolio
! pip install PyPortfolioOpt

     |████████████████████████████████| 11.6 MB 6.7 MB/s eta 0:00:01
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
     |████████████████████████████████| 91 kB 377 kB/s eta 0:00:011
     |████████████████████████████████| 52 kB 618 kB/s eta 0:00:011
  Created wheel for pyfolio: filename=pyfolio-0.9.2-py3-none-any.whl size=88667 sha256=3bb6b43b4e9c39271d8830329bcf935f43250068b4ccb07566a8732b41f73b64
  Stored in directory: /root/.cache/pip/wheels/e4/96/9b/0dfff5453e702fd780a099b7c850521099c5ec0dfafae189f9
  Created wheel for empyrical: filename=empyrical-0.5.5-py3-none-any.whl size=39762 sha256=f06d3f126436b16006843582d737c895a4b752ccdadd3fd1221bb4230c975d7a
  Stored in directory: /root/.cache/pip/wheels/d9/91/4b/654fcff57477efcf149eaca236da2fce991526cbab431bf312
Successfully built pyfolio empyrical
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
     |████████████████████████

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [11]:
import cvxopt
import numpy as np
import pandas as pd
from abc import *
from typing import NewType, Dict, List
from cvxopt import matrix
from pypfopt import risk_models as RiskModels

In [3]:
Dataframe = NewType('Dataframe', pd.core.frame.DataFrame)
Ndarray = NewType('Ndarray', np.ndarray)
Series = NewType('Series', pd.core.series.Series)
class PortfolioOptimizer(metaclass=ABCMeta):
    # prediction結果を考慮したbalanceを返してくれる
    @abstractmethod
    def optimize(prediction: Series)-> Series:
        pass

    # historyデータを一行追加する
    @abstractmethod
    def push_history(history: Series)-> None:
        pass

## データの前処理
### レート履歴の下処理¶

In [4]:
# インターフェースを実装したクラス
class ChinaOptimizer(PortfolioOptimizer):
    def __init__(self, history: Dataframe)-> None:
        self.history: Dataframe = history
        self.covariance: Dataframe = RiskModels.CovarianceShrinkage(self.history).shrunk_covariance()
        self.set_delta()
    
    def set_delta(self, column_name: str = "N225")-> None:
        self.delta: float = black_litterman.market_implied_risk_aversion(self.history[column_name])
            
    def set_omega(self)-> None:
        self.Omega: Ndarray = BlackLittermanModel.default_omega(cov_matrix = self.covariance, P = self.P, tau = 0.05)

    def push_history(self, new_date_dataframe: Dataframe)-> None:
        self.history: Dataframe = self.history.append(new_date_dataframe)
        self.covariance: Dataframe = RiskModels.CovarianceShrinkage(self.history).shrunk_covariance()
        self.set_delta()
        
    def set_Q(self, Q: Series, NonZeroPredictionRow: Ndarray)-> None:
        self.Q = np.array([Q.values[NonZeroPredictionRow]])
    
    def optimize(self, prediction: Series, Q: Series)-> Dataframe:
        P = np.eye(len(prediction)) * prediction.values
        non_zero_prediction_row = np.any(P != 0, axis=1)
        self.P = P[non_zero_prediction_row]
        self.set_Q(Q, non_zero_prediction_row)
        self.set_omega()
        bl = BlackLittermanModel(self.covariance, P = self.P, Q = self.Q, omega = self.Omega)
        rets = bl.bl_returns()
        bl.bl_weights(self.delta)
        weight = bl.clean_weights()
        return weight

In [48]:
securities = ['N225', 'GSPC', 'EZU', 'GSPTSE', 'GDAXI', 'FCHI', 'EWQ', 'EWG', 'EWC', 'GCF', 'CLF']
all_security_close = pd.DataFrame(columns = ['Date']).set_index('Date')
for security in securities:
    price = pd.read_csv("/kaggle/input/stock-price-datas/" + security + ".csv", header=0, names = ('Date', 'Open', 'High', 'Low', security, 'Adj Close', 'Volume'))
    price = price.dropna().drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis = 1).set_index('Date')
    all_security_close = pd.concat([all_security_close, price], axis=1)
all_security_close = all_security_close.dropna()
columns = ['Date']
columns.extend(all_security_close)
# all_security_close['USD'] = 1

### 予測結果の下処理  
dropnaではなく、他の方法を考える

In [49]:
securities = ['N225', 'GSPC', 'EZU', 'GSPTSE', 'GDAXI', 'FCHI', 'EWQ', 'EWG', 'EWC', 'GCF', 'CLF']
all_security_prediction = pd.DataFrame(columns = ['Date']).set_index('Date')
all_security_twenty = pd.DataFrame(columns = ['Date']).set_index('Date')
for security in securities:
    prediction = pd.read_csv("/kaggle/input/indexprediction/" + security + ".csv", header=0, names = ('Date', security + '_Prediction', 'Ans', security + '_Twenty'))
    prediction = prediction.dropna().drop(['Ans'], axis = 1).drop(range(20)).set_index('Date')
    all_security_prediction = pd.concat([all_security_prediction, prediction[security + '_Prediction']], axis=1)
    all_security_prediction.rename(columns={security + '_Prediction': security}, inplace=True)
    # ラベル予測→パーセンテージの処理
    # ここはどうにかすべき
    all_security_prediction.loc[all_security_prediction[security] == 1, security] = 0.005
    all_security_prediction.loc[all_security_prediction[security] == -1, security] = -0.005
    all_security_prediction.loc[all_security_prediction[security] == 2, security] = 0.02
    all_security_prediction.loc[all_security_prediction[security] == -2, security] = -0.02
    all_security_twenty = pd.concat([all_security_twenty, prediction[security + '_Twenty']], axis=1)
    all_security_twenty.rename(columns={security + '_Twenty': security}, inplace=True)
# twentyの計上方法に従って　最初の20行を削除
all_security_prediction = all_security_prediction.dropna()
all_security_twenty = all_security_twenty.dropna()
all_security_prediction

,N225,GSPC,EZU,GSPTSE,GDAXI,FCHI,EWQ,EWG,EWC,GCF,CLF
2019-04-04,0.005,0.000,-0.005,-0.005,-0.005,-0.005,-0.005,-0.005,0.005,0.005,-0.020
2019-04-05,0.005,0.000,-0.005,0.005,0.005,0.005,-0.005,-0.005,-0.005,0.005,0.005
2019-04-08,0.005,0.000,0.020,-0.005,0.005,0.005,-0.005,0.005,-0.005,0.005,0.005
2019-04-09,0.005,0.000,0.020,-0.005,-0.005,0.005,0.000,0.005,0.020,0.020,0.020
2019-04-10,0.005,0.000,0.005,0.005,0.005,-0.005,0.000,0.020,0.020,0.020,-0.020
...,...,...,...,...,...,...,...,...,...,...,...
2020-08-04,0.005,0.000,-0.020,0.000,0.005,0.020,0.020,0.020,0.020,0.000,-0.020
2020-08-05,0.005,0.000,0.020,0.000,0.005,0.020,0.020,0.020,0.020,0.000,0.005
2020-08-06,0.005,0.000,0.020,-0.005,0.020,0.020,0.020,0.020,0.020,0.000,-0.020
2020-08-07,0.020,0.005,0.020,-0.020,0.020,0.020,0.020,0.020,0.020,0.000,-0.020


In [75]:
covariance: Dataframe = RiskModels.CovarianceShrinkage(all_security_close[:all_security_prediction.iloc[0].name]).shrunk_covariance()
Sigma = matrix(covariance.to_numpy())
r = matrix(all_security_close.loc[all_security_prediction.iloc[0].name].values.tolist())
# 合計が1になる制約
A = matrix(np.ones(len(securities)).astype(np.float)).T
b = matrix(np.array([1.0]))

G = matrix(np.vstack([np.eye(len(securities)),-1 * np.eye(len(securities))]).astype(np.float))
# G = matrix(np.eye(len(securities)).astype(np.float))
h = matrix(np.hstack([np.ones(len(securities)), np.zeros(len(securities))]))
# h = matrix(np.hstack(np.ones(len(securities))))

sol = cvxopt.solvers.qp(Sigma,r, A=A, b=b,G=G, h=h)
# sol = cvxopt.solvers.qp(Sigma,r, A=A, b=b)
print(sol)
print(sol['x'])
print(sol['primal objective'])


     pcost       dcost       gap    pres   dres
 0: -2.8668e+08 -3.7699e+06  1e+09  1e+04  1e-16
 1: -2.9593e+06 -8.1856e+04  1e+07  1e+02  2e-14
 2: -5.8708e+04 -7.9471e+04  3e+05  2e+00  1e-14
 3: -3.5869e+03 -3.0756e+04  4e+04  1e-01  1e-14
 4:  2.3410e+02 -1.3260e+03  2e+03  1e-03  2e-15
 5:  6.4978e+01 -8.6410e-01  7e+01  5e-05  4e-16
 6:  3.2761e+01  2.5225e+01  8e+00  4e-06  3e-16
 7:  2.8481e+01  2.7445e+01  1e+00  3e-07  1e-16
 8:  2.8018e+01  2.7896e+01  1e-01  4e-08  1e-16
 9:  2.7942e+01  2.7940e+01  2e-03  3e-10  2e-16
10:  2.7941e+01  2.7941e+01  2e-05  3e-12  3e-16
Optimal solution found.
{'x': <11x1 matrix, tc='d'>, 'y': <1x1 matrix, tc='d'>, 's': <22x1 matrix, tc='d'>, 'z': <22x1 matrix, tc='d'>, 'status': 'optimal', 'gap': 1.826674806005817e-05, 'relative gap': 6.537675018742615e-07, 'primal objective': 27.94075854554825, 'dual objective': 27.940740412593033, 'primal infeasibility': 3.1780926159193596e-12, 'dual infeasibility': 2.7499318185069596e-16, 'primal slack': 